*Файл `admin.py` в Django предназначен для регистрации моделей в административной панели Django, чтобы можно было управлять данными через удобный веб-интерфейс. Он позволяет кастомизировать, как модели будут отображаться, какие поля будут доступны для редактирования, какие фильтры и действия можно будет выполнять, а также многое другое. Админка — это мощный инструмент, который можно настроить для удобного управления данными.*

### Основные шаги работы с `admin.py`:

1. **Импортируем необходимые модели**:
   Чтобы начать работать с админкой, нужно импортировать модель, которую вы хотите зарегистрировать.
   ```python
   from django.contrib import admin
   from .models import Post
   ```

2. **Регистрация модели в админке**:
   Чтобы модель появилась в административной панели, её нужно зарегистрировать. Для этого используется метод `admin.site.register()`.

   ```python
   admin.site.register(Post)
   ```

   Это минимальный способ регистрации, который позволяет отображать объекты модели в административной панели.

---

## Расширенная настройка админки

В большинстве случаев вам понадобится больше настроек для управления отображением и функциональностью административной панели. Для этого вы можете создавать классы настроек админки, наследуя их от `admin.ModelAdmin`.

### 1. **Класс ModelAdmin**
Класс `ModelAdmin` предоставляет вам возможность детализировано настраивать, как модель будет отображаться в админке.

#### Основные параметры:

1. **`list_display`**:
   Определяет, какие поля модели будут отображаться в списке объектов на главной странице админки.
   
   ```python
   class PostAdmin(admin.ModelAdmin):
       list_display = ('title', 'author', 'published_date', 'status')
   ```

2. **`list_filter`**:
   Добавляет возможность фильтрации объектов по указанным полям в интерфейсе админки.

   ```python
   class PostAdmin(admin.ModelAdmin):
       list_display = ('title', 'author', 'published_date', 'status')
       list_filter = ('status', 'published_date')
   ```

3. **`search_fields`**:
   Позволяет добавлять поле поиска, чтобы администратор мог искать объекты по указанным полям.

   ```python
   class PostAdmin(admin.ModelAdmin):
       list_display = ('title', 'author', 'published_date', 'status')
       search_fields = ('title', 'content')
   ```

4. **`ordering`**:
   Определяет, по каким полям объекты будут сортироваться в списке по умолчанию.

   ```python
   class PostAdmin(admin.ModelAdmin):
       list_display = ('title', 'author', 'published_date', 'status')
       ordering = ('-published_date',)
   ```

5. **`list_editable`**:
   Позволяет редактировать поля прямо в списке объектов без необходимости перехода на страницу редактирования объекта.

   ```python
   class PostAdmin(admin.ModelAdmin):
       list_display = ('title', 'author', 'status')
       list_editable = ('status',)
   ```

6. **`prepopulated_fields`**:
   Полезно для автоматической генерации значений одного поля на основе другого. Часто используется для полей `SlugField`, которые генерируются из заголовков.

   ```python
   class PostAdmin(admin.ModelAdmin):
       prepopulated_fields = {'slug': ('title',)}
   ```

7. **`readonly_fields`**:
   Задает поля, которые будут доступны только для чтения. Эти поля нельзя будет редактировать в админке.

   ```python
   class PostAdmin(admin.ModelAdmin):
       readonly_fields = ('created_at', 'updated_at')
   ```

8. **`fields`**:
   Определяет, в каком порядке и какие поля будут отображаться на странице редактирования объекта.

   ```python
   class PostAdmin(admin.ModelAdmin):
       fields = ('title', 'slug', 'content', 'status', 'published_date')
   ```

9. **`fieldsets`**:
   Этот параметр позволяет разделить форму редактирования объекта на секции для лучшей организации.

   ```python
   class PostAdmin(admin.ModelAdmin):
       fieldsets = (
           (None, {
               'fields': ('title', 'slug')
           }),
           ('Content', {
               'fields': ('content',)
           }),
           ('Publishing Info', {
               'fields': ('status', 'published_date')
           }),
       )
   ```

10. **`inlines`**:
    Позволяет редактировать связанные объекты прямо на странице редактирования основной модели. Это удобно для моделей с отношениями ForeignKey или ManyToMany.

    Например, если у вас есть модель `Comment`, связанная с моделью `Post`, можно настроить редактирование комментариев прямо внутри поста:

    ```python
    from .models import Comment

    class CommentInline(admin.TabularInline):
        model = Comment
        extra = 1  # Сколько пустых полей для создания комментариев будет показываться

    class PostAdmin(admin.ModelAdmin):
        inlines = [CommentInline]
    ```

    Есть два типа отображения связанных объектов:
    - `TabularInline`: отображает связанные объекты в виде таблицы.
    - `StackedInline`: отображает связанные объекты в виде вертикальных блоков.

---

### 2. **Кастомные действия в админке**

Админка позволяет добавлять свои собственные действия для работы с объектами. Например, можно добавить возможность массового изменения статуса постов.

#### Пример кастомного действия:
```python
def make_published(modeladmin, request, queryset):
    queryset.update(status='published')

make_published.short_description = "Отметить как опубликованное"

class PostAdmin(admin.ModelAdmin):
    list_display = ('title', 'author', 'status')
    actions = [make_published]
```

Теперь в админке появится новое действие, которое можно применить к выделенным объектам.

---

### 3. **Кастомизация формы редактирования**

Django позволяет кастомизировать формы, которые используются для создания и редактирования объектов в админке. Для этого нужно создать собственные формы, используя `forms.ModelForm`, а затем указать их в классе `ModelAdmin`.

#### Пример кастомной формы:
```python
from django import forms
from .models import Post

class PostForm(forms.ModelForm):
    class Meta:
        model = Post
        fields = ['title', 'content', 'status']

    def clean_title(self):
        title = self.cleaned_data.get('title')
        if "spam" in title:
            raise forms.ValidationError("Нельзя использовать слово 'spam' в заголовке!")
        return title

class PostAdmin(admin.ModelAdmin):
    form = PostForm
```

Таким образом, кастомная логика для проверки данных будет выполнена при сохранении объектов через админку.

---

### 4. **Админка и отношения "многие ко многим"**

Когда модель содержит поле ManyToMany, оно не отображается по умолчанию в форме редактирования объекта. Вы можете использовать виджет "Filter horizontal" или "Filter vertical" для отображения этих полей.

#### Пример:
```python
class PostAdmin(admin.ModelAdmin):
    filter_horizontal = ('tags',)  # Для горизонтального отображения
    filter_vertical = ('tags',)    # Для вертикального отображения
```

---

### 5. **Регистрация нескольких моделей с одинаковыми настройками**

Если вам нужно зарегистрировать несколько моделей с одинаковыми настройками админки, вы можете повторно использовать один и тот же класс `ModelAdmin`.

#### Пример:
```python
admin.site.register(Post, PostAdmin)
admin.site.register(Comment, PostAdmin)  # Используем тот же класс настроек
```

---

### 6. **Подключение сторонних приложений к админке**

Сторонние Django-приложения часто содержат свои собственные модели, которые можно подключить к административной панели. Например, приложение `django.contrib.auth` предоставляет модель `User`, которая уже зарегистрирована в админке, но вы можете кастомизировать её.
